# B6A Technical Challenge - Sentiment Analysis #

## Get the Data ##
Let's take advantage of pandas to read the csv file and convert it to pandas.DataFrame.

In [1]:
import pandas as pd

We will use read_csv() to read csv file, decode the tweets using 'ISO-8859-1' standard and rename every column.

In [2]:
tweets = pd.read_csv('../training.1600000.processed.noemoticon.csv',
        encoding = "ISO-8859-1", names=['polarity','tweet_id','date',
        'query','user_id','tweet'])

In [3]:
# roughly check the structure of the dataset
tweets.head()

,polarity,tweet_id,date,query,user_id,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Since we are only interested in polarity and tweet, we take out polarity and tweet and put them into numpy array.

In [4]:
train_set = tweets['tweet'].values
score_set = tweets['polarity'].values

## Text Pre-processing ##
Logistic regression algorithm requires numerical vectors to perform classification tasks. We are going to convert every tweet into a vector consisting of different words. We are going to perform the following precedures to pre-process tweets:

1. remove non-letters
2. restore contractions
3. remove punctuation
4. tokenize tweet
4. remove stopwords
5. stem words

We first import some packages for test pre-processing

In [5]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

We define some constants for text pre-processing.

In [6]:
ENGLISH_STOPWORDS = set(stopwords.words('english'))
CONTRACTIONS = [
    (r'\'m', ' am'),
    (r'\'ll', ' will'),
    (r'\'s', ' s'),
    (r'\'d', ' had'),
    (r'aren\'t', 'are not'),
    (r'can\'t', 'can not'),
    (r'couldn\'t', 'could not'),
    (r'daren\'t', 'dare not'),
    (r'didn\'t', 'did not'),
    (r'doesn\'t', 'does not'),
    (r'don\'t', 'do not'),
    (r'isn\'t', 'is not'),
    (r'hasn\'t', 'has not'),
    (r'haven\'t', 'have not'),
    (r'hadn\'t', 'had not'),
    (r'mayn\'t', 'may not'),
    (r'mightn\'t', 'might not'),
    (r'mustn\'t', 'must not'),
    (r'needn\'t', 'need not'),
    (r'oughtn\'t', 'ought not'),
    (r'shan\'t', 'shall not'),
    (r'shouldn\'t', 'should not'),
    (r'wasn\'t', 'was not'),
    (r'weren\'t', 'were not'),
    (r'won\'t', 'will not'),
    (r'wouldn\'t', 'would not'),
    (r'ain\'t', 'am not')
]

We define a function to convert each tweet to a list of words.

In [7]:
"""
this function process split each tweet into a list of words.

args:
    tweet: a tweet (string)

returns:
    a list of words (list)
"""
def process_tweet(tweet):
    # remove non-letters
    tweet = re.sub("[^a-zA-Z]"," ", tweet)

    # restore contractions 
    for pattern in CONTRACTIONS:
        tweet = re.sub(pattern[0], pattern[1], tweet)
    
    # remove punctuation
    nopunc = [char for char in tweet if char not in string.punctuation]
    nopunc = ''.join(nopunc) # join characters together
    
    # convert words to lower case and tokenize them
    words = nopunc.lower().split()
    
    # remove stopwords
    words = [word for word in words if word not in ENGLISH_STOPWORDS]
    
    # stemming each word
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    
    return words

## Compute Featue Matrix ##
In this part, we will convert text to numbers using the Bag-of-Words model, or BoW. This model ignores the order information and focuses on the occurrence of words in a document. This can be done by assigning each word a unique number. Then any document we see can be encoded as a fixed-length vector with the length of the vocabulary of known words. The value in each position in the vector could be filled with a count or frequency of each word in the encoded document. To use BoW, Sciki-Learn provides three different schemes, **CountVectorizer**, **TfidfVectorizer**, **HashingVectorizer**. <br> In order to better leverage our logistic regression algorithm, we will use CountVectorizer for this project. Since there are so many features, we can expect a lot of zero counts for the presence of that word in that document. Because of this, SciKit Learn will output a **Sparse Matrix**.

We first import some useful packages.

In [8]:
import numpy as np
from scipy import sparse, stats
from sklearn.feature_extraction.text import CountVectorizer

We define maximum amount of features we want. We will build a vocabulary that only consider the top MAX_FEATURES ordered by **term frequency** across the corpus.

In [9]:
MAX_FEATURES = 5000

We define a function to train the bag-of-word transformer that is used to transform tweets to feature matrix.

In [10]:
"""
this function trains CountVectorizer model with the trainning set and extract
the features we want.

args:
    tweet_arr: the whole trainning set (numpy.array)
returns:
    bow_transformer: bag-of-word transformer (scipy.sparse.csr_matrix)
"""
def make_bow_transformer(tweet_arr):
    # this function fits the countvectorizer
    bow_transformer = CountVectorizer(tokenizer=lambda doc: doc, 
        lowercase=False, max_features=MAX_FEATURES).fit(tweet_arr)

    return bow_transformer

This function computes the NxM feature matrix where N is the number of tweets and M is the number of the features.<br> In addition, we add one more feature that is set as 1 for every tweet to compute the **bias** term, **b**.

In [11]:
"""
this function transforms trainning set to feature matrix using 
bag-of-word model. it computes teh NxM matrix where N is the 
number of tweets and M is the number of the features.

args:
    tweet_arr: trainning set (numpy.array)
    bow_transformer: bag-of-word transformer (scipy.sparse.csr_matrix)
returns:
    tweet_bow: feature matrix (scipy.sparse.csr_matrix)
"""
def compute_feature_matrix(tweet_arr, bow_transformer):
    # vetorize tweets
    tweet_bow = bow_transformer.transform(tweet_arr)

    # to compute bias, add one more feature that has set to be 1 for every tweet
    bias_vector = np.ones(tweet_bow.shape[0])[:,None]
    tweet_bow = sparse.hstack((tweet_bow,bias_vector),
     dtype=tweet_bow.dtype).toarray()

    return tweet_bow

## Logistic Regression ##
After pre-process the tweets, we can work on out logistic regression module now.

### Sigmoid Function ###

In [12]:
"""
sigmoid function

args:
    a: a matrix (numpy.array)

returns:
    a matrix (numpy.array)
"""
def sigmoid(a):
    return 1.0 / (1.0 + np.exp(-1.0 * a))

### Cost Function ###

We define the probability function calculate the probablity of the sentiment of each tweet being categorized as 1.

In [13]:
"""
function to calculate the probability

args:
    feature: feature of a single tweet (numpy.array)
    weight: weight vector (numpy.array)

returns:
    a matrix (numpy.array)
"""
def prob_function(feature, weight):
    # np dot matrix multiplication
    return sigmoid(np.dot(feature, weight))

We then define the cost function to calculate the cost value.

In [14]:
"""
cost function

args:
	w: weight vector (numpy.array)
	x: trainning set (numpy.array)
	y: score set (numpy.array)
	
returns:
	the cost value (float)
"""
def cost(w, x, y):
    [n,m] = x.shape
    cost_value = 0

    for i in range(n):
        cost_value += (y[i] * np.log(prob_function(x[i], w)) +
                       (1.0 - y[i]) * np.log(1.0 - prob_function(x[i], w)))
    
    # since the sign of cost value in the gradient descent formula
    # in negative, so we times a -1.0 to the result
    return -1.0 * cost_value

### Gradient Descent ###

We first define some constants for gradient descent.

In [15]:
SPEED = 0.001
MAX_ITERATIONS = 2

We define a function for stochastic gradient descent. We will randomly shuffle the data set at the beginning of each epoch. For this project, we won't check whether the model converges or not after each epoch. Instead, we will make the model only run for 2 epochs.

In [16]:
"""
Gradient descent (stochastic) to find the local minimum of cost function. 

args:
    train_set: the trainning input (numpy.array)
    score_set: the trainning output (numpy.array)

returns:
    the weight vector (numpy.array)
"""
def gradient_descent(train_set, score_set):
    # get the shape of the train set
    [row, column] = train_set.shape
    
    # initialize weight vector
    weight_vector = np.zeros(column)
    
    # variables to show stats
    iteration = 0
    last_cost_val = 0

    while (iteration < MAX_ITERATIONS):
        # shuffle training set at the beginning of each epoch
        indices = np.random.permutation(train_set.shape[0])
        train_set, score_set = train_set[indices], score_set[indices]
        
        # stochastic gradient descent
        for i in range(row):
            prediction = prob_function(train_set[i], weight_vector)
            error = score_set[i] - prediction
            weight_vector = weight_vector + SPEED * train_set[i].transpose() * error
        
        cost_val = cost(weight_vector, train_set, score_set)
        
        # calculate the accuracy of current weight
        valid = 0
        for i in range(row):
            if prob_function(weight_vector, train_set[i]) >= 0.5:
                prediction = 1
            else:
                prediction = 0
            
            if prediction == score_set[i] :
                valid += 1
        
        percent = 100.0 * valid / row
        
        iteration += 1
        
        # show stats
        print("iteration %d..." % iteration)
        print('cost value: %.4f' % cost_val)
        print('well-classified tweets: {0} / {1} ({2}%)'.format(valid, row, percent))
        print()

    return weight_vector

### Train Logistic Regression Model ###

We first define a constant for trainning.

In [17]:
NUM_PER_GROUP = 100000

We define a function to help us seperate trainning set into several groups.

In [18]:
import itertools

In [19]:
"""
this function helps to split an array into several groups.

args:
	n:	the number of items in each group (int)
	iterable: an iterable object (object)

returns:
	a tuple consisting tuples with same amoumt of items (tuple) 
"""
def grouper(n, iterable):
    args = [iter(iterable)] * n
    return itertools.zip_longest(*args, fillvalue=None)

We define a function to train the logistic regression model. 

In this part, I mainly conquered 2 challenges:
1. **the challenge of handling large amount of data set**. If we compute the whole 1.6 million tweets at once, one problem is that we cannot have too many features because the computing process requires huge amount of memory. In order to incorporate more features, we shuffle the training set first and then split the 1.6 million tweets into 16 groups with each group having 10,0000 tweets. Then we compute the weight vectors of these 16 groups seperately. At the end, a final weight vector is generated by calculating the weighted average of the 16 weight vectors.
2. **the challenge of setiing the initial weight vector**. Because the instruction requires us to run 2 epochs, we cannot guarantee that the model actually converges by only runing 2 epochs. Thus, choosing the initial weight vector is critical for obtaining a good model. I have tried to fill the intial weight vector with all 0s, 1s and numbers uniformly distributed between -1 and 1. It turned out that having initial vector set as all 0s trains a better model.

In [20]:
"""
this functino trains the model.

args:
    train_set: the trainning input (numpy.array)
    score_set: the trainning output (numpy.array)
returns:
    weight_vector: trained weight vector (numpy.array)
    bow_transformer: bag-of-word transformer (scipy.sparse.csr_matrix)
"""
def train(train_set,score_set):
    # tokenize tweets here
    tweet_list = []
    for tweet in train_set:
        tweet_list.append(process_tweet(tweet))

    # convert tweet_list to numpy array
    tweet_arr = np.array(tweet_list)

    # train bag-of-words prob_function
    bow_transformer = make_bow_transformer(tweet_arr)

    # to store weight vectors for each group
    weight_vector_list = []

    # index to split trainning set into n group
    group_index_list = []

    # get the number of tweets in trainning set
    num_items = len(train_set)

    # split trainning data into several groups
    group_index_list = list(grouper(NUM_PER_GROUP,
     np.random.permutation(num_items)))

    print('==== Start training...')

    for i in range(len(group_index_list)):
        # quit if it is not a complete group
        if len(group_index_list[i]) < NUM_PER_GROUP:
            break

        print("group # ", i+1)
        
        # compute feature matrix
        feature_array = compute_feature_matrix(
            tweet_arr[list(group_index_list[i])], bow_transformer)
        
        # transform score set
        score_matrix = score_set[list(group_index_list[i])]
        score_array = np.transpose(np.array(score_matrix))
        
        weight_vector_part = gradient_descent(feature_array, score_array)
        weight_vector_list.append(weight_vector_part)
    
    # compute weighted average of the weight vector of each group
    weight_vector = np.mean(weight_vector_list, axis=0)
    
    print('==== Done')
    
    return weight_vector, bow_transformer

### Invoke Training ###

In [21]:
# modify positive sentiment score from 4 to 1
score_set[score_set == 4] = 1

# shuffle dataset
indices = np.random.permutation(score_set.shape[0])
x_train, y_train = train_set[indices], score_set[indices]

# train the model
print ('===== training model...')
weight_vector, bow_transformer = train(train_set, score_set)
print ('===== model trained!')

===== training model...
==== Start training...
group #  1
iteration 1...
cost value: 60908.4799
well-classified tweets: 71860 / 100000 (71.86%)

iteration 2...
cost value: 58164.3824
well-classified tweets: 72882 / 100000 (72.882%)

group #  2
iteration 1...
cost value: 60965.8512
well-classified tweets: 71005 / 100000 (71.005%)

iteration 2...
cost value: 58176.4536
well-classified tweets: 72726 / 100000 (72.726%)

group #  3
iteration 1...
cost value: 60864.3668
well-classified tweets: 71391 / 100000 (71.391%)

iteration 2...
cost value: 58154.4848
well-classified tweets: 72885 / 100000 (72.885%)

group #  4
iteration 1...
cost value: 61042.8048
well-classified tweets: 71448 / 100000 (71.448%)

iteration 2...
cost value: 58311.7815
well-classified tweets: 72458 / 100000 (72.458%)

group #  5
iteration 1...
cost value: 60878.2248
well-classified tweets: 71610 / 100000 (71.61%)

iteration 2...
cost value: 58111.9396
well-classified tweets: 72728 / 100000 (72.728%)

group #  6
iteration

## Outputs ##
After running this jupyter notebook on AWS EC2, we have the following outputs.

In [22]:
# sort the weight vector
w_output = weight_vector.argsort()

print("bottom 5 words by weight: ")
for i in w_output[:5]:
    if (i != len(w_output) - 1):
        print("---->", bow_transformer.get_feature_names()[i])

print("top 5 words by weight: ")
for i in w_output[:-6:-1]:
    if (i != len(w_output) - 1):
        print("---->", bow_transformer.get_feature_names()[i])

print("the value of the bias: ")
print("---->", weight_vector[-1])

bottom 5 words by weight: 
----> miss
----> sad
----> wish
----> hate
----> sorri
top 5 words by weight: 
----> thank
----> love
----> good
----> great
----> happi
the value of the bias: 
----> 0.0393152994258
